<center><h1>Data Manipulation: Forming a Weekly Time Series<br><p style="font-size:8">(Data Manipulation and ARIMA Modeling with Pyramid)</p></h1></center>

# 0. Prelim

## 0.1 Packages

In [11]:
from pathlib import Path
import numpy as np
import pandas as pd

## 0.2 Paths

In [12]:
DATA_FOLDER = Path("../../../data")
RAW_DATA_FOLDER = DATA_FOLDER / "raw"

# 1. Extract Data

In [13]:
# A. Extract
df_bookings = pd.read_csv(RAW_DATA_FOLDER / "H1.csv", parse_dates=['ReservationStatusDate'])

# B. Transform
df_bookings['ArrivalDate'] = pd.to_datetime(
    df_bookings[['ArrivalDateYear', 'ArrivalDateMonth', 'ArrivalDateDayOfMonth']].astype(str)\
        .agg("-".join, axis=1)
)
df_bookings['ArrivalWeekDay']= df_bookings['ArrivalDate'].dt.strftime('%A')
# df_bookings.drop(
#     columns=['ArrivalDateYear', 'ArrivalDateMonth', 'ArrivalDateDayOfMonth'],
#     inplace=True
#     )

# C. Display
print("Descriptive Statistics:")
print("-----------------------")
print(f"Shape : {df_bookings.shape}")
print("\n\n")
print(f"Columns:")
print(f"--------")
display(df_bookings.dtypes)
print("\n\n")
display(df_bookings)

Descriptive Statistics:
-----------------------
Shape : (40060, 33)



Columns:
--------


IsCanceled                              int64
LeadTime                                int64
ArrivalDateYear                         int64
ArrivalDateMonth                       object
ArrivalDateWeekNumber                   int64
ArrivalDateDayOfMonth                   int64
StaysInWeekendNights                    int64
StaysInWeekNights                       int64
Adults                                  int64
Children                                int64
Babies                                  int64
Meal                                   object
Country                                object
MarketSegment                          object
DistributionChannel                    object
IsRepeatedGuest                         int64
PreviousCancellations                   int64
PreviousBookingsNotCanceled             int64
ReservedRoomType                       object
AssignedRoomType                       object
BookingChanges                          int64
DepositType                       

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate,ArrivalDate,ArrivalWeekDay
0,0,342,2015,July,27,1,0,0,2,0,...,NULL,0,Transient,0.00,0,0,Check-Out,2015-07-01,2015-07-01,Wednesday
1,0,737,2015,July,27,1,0,0,2,0,...,NULL,0,Transient,0.00,0,0,Check-Out,2015-07-01,2015-07-01,Wednesday
2,0,7,2015,July,27,1,0,1,1,0,...,NULL,0,Transient,75.00,0,0,Check-Out,2015-07-02,2015-07-01,Wednesday
3,0,13,2015,July,27,1,0,1,1,0,...,NULL,0,Transient,75.00,0,0,Check-Out,2015-07-02,2015-07-01,Wednesday
4,0,14,2015,July,27,1,0,2,2,0,...,NULL,0,Transient,98.00,0,1,Check-Out,2015-07-03,2015-07-01,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40055,0,212,2017,August,35,31,2,8,2,1,...,NULL,0,Transient,89.75,0,0,Check-Out,2017-09-10,2017-08-31,Thursday
40056,0,169,2017,August,35,30,2,9,2,0,...,NULL,0,Transient-Party,202.27,0,1,Check-Out,2017-09-10,2017-08-30,Wednesday
40057,0,204,2017,August,35,29,4,10,2,0,...,NULL,0,Transient,153.57,0,3,Check-Out,2017-09-12,2017-08-29,Tuesday
40058,0,211,2017,August,35,31,4,10,2,0,...,NULL,0,Contract,112.80,0,1,Check-Out,2017-09-14,2017-08-31,Thursday


## 2. Weekly Aggregation

In [20]:
# A. Create Aggregation
df_weekly_cancelations = df_bookings.resample("W", on='ArrivalDate', closed='left')['IsCanceled'].sum().reset_index()
df_weekly_cancelations['Date (Year and Week Number)'] = df_weekly_cancelations['ArrivalDate'].dt.strftime("%Y%-U")

# B. Display
print("Descriptive Statistics:")
print("-----------------------")
print(f"Shape : {df_weekly_cancelations.shape}")
print(f"Date Range: {df_weekly_cancelations['ArrivalDate'].min() : %Y-%m-%d} - {df_weekly_cancelations['ArrivalDate'].max() : %Y-%m-%d}")
print("\n\n")
print(f"Columns:")
print(f"--------")
display(df_weekly_cancelations.dtypes)
print("\n\n")
print(f"Quality Check:")
print(f"--------------")
display(df_weekly_cancelations.loc[df_weekly_cancelations['Date (Year and Week Number)'].isin(['201527', '201528', '201529']), ['Date (Year and Week Number)', 'IsCanceled']])
display(df_weekly_cancelations)

Descriptive Statistics:
-----------------------
Shape : (114, 3)
Date Range:  2015-07-05 -  2017-09-03



Columns:
--------


ArrivalDate                    datetime64[ns]
IsCanceled                              int64
Date (Year and Week Number)            object
dtype: object




Quality Check:
--------------


,Date (Year and Week Number),IsCanceled
0,201527,41
1,201528,48
2,201529,87


,ArrivalDate,IsCanceled,Date (Year and Week Number)
0,2015-07-05,41,201527
1,2015-07-12,48,201528
2,2015-07-19,87,201529
3,2015-07-26,74,201530
4,2015-08-02,101,201531
...,...,...,...
109,2017-08-06,173,201732
110,2017-08-13,140,201733
111,2017-08-20,182,201734
112,2017-08-27,143,201735
